In [30]:
import numpy as np
import pandas as pd

In [31]:
# lol = pd.read_csv("./2021_LoL_esports.csv")
lol = pd.read_csv("./2021_LoL_esports (cópia).csv")
lol.replace(np.nan, 0, inplace= True)
lol['Firstblood'] = lol['firstblood'].map({1.0:'fb' , 0.0:'no_fb'})
lol['Result'] = lol['result'].map({1:'win' , 0:'lose'})

lol.loc[lol.position=='team',('playername','champion')]='team'
# lol.loc[lol.position==0,('ban1','ban2','ban3','ban4','ban5')]='DC'

# lol['Playername'] = lol['playername'].map({0:'team'})

lol = lol.drop(columns=['patch','gameid','url','datacompleteness','year','date','split','playerid','teamid','opp_csat15','golddiffat15','xpdiffat15','csdiffat15','killsat15',
    'assistsat15','deathsat15','opp_killsat15','opp_assistsat15','opp_deathsat15','assistsat10','deathsat10','opp_killsat10','opp_assistsat10','opp_deathsat10','goldat15',
    'xpat15','csat15','opp_goldat15','opp_xpat15','goldat10','xpat10','csat10','opp_goldat10','opp_xpat10','opp_csat10','golddiffat10','xpdiffat10','csdiffat10','killsat10',
    'monsterkills','monsterkillsownjungle','monsterkillsenemyjungle','teamdeaths','doublekills','triplekills','quadrakills','pentakills','teamkills','gamelength','team kpm',
    'elementaldrakes','opp_elementaldrakes','chemtechs','hextechs','dragons (type unknown)','turretplates','opp_turretplates','ckpm','dpm','damageshare','damagetakenperminute',
    'damagemitigatedperminute','wpm','wcpm','vspm','earned gpm','earnedgoldshare','gspd','cspm',
    
    'damagetochampions','total cs','inhibitors','opp_inhibitors','wardsplaced','wardskilled','controlwardsbought','visionscore','totalgold','earnedgold','goldspent','opp_heralds',
    'firstbaron','barons','opp_barons','firsttower','towers','opp_towers','firstmidtower','firsttothreetowers','minionkills','dragons','opp_dragons','infernals','mountains',
    'clouds','oceans','elders','opp_elders','firstherald','heralds','kills','deaths','assists','firstbloodkill','firstbloodassist','firstbloodvictim','firstdragon',

    'firstblood','result','participantid','playoffs','game',
    'ban1','ban2','ban3','ban4','ban5',
    'league','playername'#,'position'
    ])

# filtro = lol['game']<74006

# lol = lol[filtro]

# lol 0 lol.drop()
# lol.head(12)

# lol.to_csv("./lol.csv")
# lol = pd.read_csv("./lol.csv",header = None)
lol.head(10)


,side,position,teamname,champion,Firstblood,Result
0,Blue,top,DWG KIA,Mordekaiser,fb,win
1,Blue,jng,DWG KIA,Graves,no_fb,win
2,Blue,mid,DWG KIA,Zoe,no_fb,win
3,Blue,bot,DWG KIA,Miss Fortune,no_fb,win
4,Blue,sup,DWG KIA,Galio,no_fb,win
5,Red,top,Nongshim RedForce,Ornn,no_fb,lose
6,Red,jng,Nongshim RedForce,Kindred,no_fb,lose
7,Red,mid,Nongshim RedForce,Orianna,no_fb,lose
8,Red,bot,Nongshim RedForce,Ezreal,no_fb,lose
9,Red,sup,Nongshim RedForce,Leona,no_fb,lose


In [32]:
def Remove(_list, value):
    return list(filter(lambda x: x != value, _list))

In [33]:
lista_lol = []

for index, row in lol.iterrows():
    L_lol = row.values.tolist()
    L_lol = Remove(L_lol,0)
    lista_lol.append(L_lol)

print(lista_lol[0]) 

['Blue', 'top', 'DWG KIA', 'Mordekaiser', 'fb', 'win']


In [34]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(lista_lol).transform(lista_lol)
lol = pd.DataFrame(te_ary, columns=te.columns_)

lol.head(1)

,100 Thieves,100 Thieves Academy,100 Thieves Next,1907 Fenerbahçe Esports,404 Multigaming e.V.,4Elements Esports,4Elements Scuttle Squad,5 Ronin,5 Ronin Academy,7more7 Pompa Team,...,piratesports,sup,team,top,unknown team,win,x25 Esports,İstanbul Wildcats,İstanbul Wildcats Academy,ŠAIM SE SuppUp
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False


In [35]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(lol, min_support = 0.01, use_colnames = True)
frequent_itemsets.sort_values(by=['support'], ascending = False)

,support,itemsets
42,0.750798,(no_fb)
40,0.500041,(lose)
21,0.500000,(Red)
3,0.500000,(Blue)
47,0.499959,(win)
...,...,...
53,0.010151,"(Azir, mid)"
50,0.010124,"(win, Alistar)"
163,0.010111,"(win, sup, Alistar)"
96,0.010057,"(no_fb, Olaf)"


In [38]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
rules = rules.sort_values(by=['lift'], ascending = False).drop(['antecedent support', 'consequent support', 'leverage', 'conviction'], axis=1)

rules.to_csv("./lol.csv")

rules.head(50)

,antecedents,consequents,support,confidence,lift
216,(Gnar),"(win, top)",0.010287,0.529760,6.357634
240,(Leona),"(Red, sup)",0.012551,0.526510,6.318117
124,(Alistar),"(win, sup)",0.010111,0.524912,6.299458
213,(Gnar),"(no_fb, top)",0.017282,0.890010,6.287226
494,"(Kai'Sa, lose)","(bot, no_fb)",0.012138,0.847570,6.234162
121,(Alistar),"(no_fb, sup)",0.015639,0.811930,6.225483
231,(Kai'Sa),"(bot, win)",0.015464,0.518703,6.224944
255,(Orianna),"(no_fb, mid)",0.015470,0.863122,6.184385
493,"(Kai'Sa, no_fb)","(bot, lose)",0.012138,0.510227,6.122231
127,(Aphelios),"(bot, no_fb)",0.013652,0.830251,6.106776
